In [1]:
from efficientnet_pytorch import EfficientNet
from torchvision import transforms
import torch

/home/moonlab/anaconda3/envs/karthik/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [18]:
model_effi = EfficientNet.from_pretrained('efficientnet-b0')

Loaded pretrained weights for efficientnet-b0


In [16]:
input = torch.randn(3, 244, 244)
input = input.unsqueeze(0)
with torch.no_grad():
    output = model_effi(input)
print(output.shape)

torch.Size([1, 1000])


In [13]:
from bird_view.models.resnet import get_resnet

model_resnet, _ = get_resnet(model_name='resnet34', pretrained=True, input_channel=3, bias_first=False)

3
Loading ResNet weights from : https://download.pytorch.org/models/resnet34-333f7ec4.pth


In [14]:
with torch.no_grad():
    output = model_resnet(input)
print(output.shape)

torch.Size([1, 512, 8, 8])


In [24]:
from torchsummary import summary
summary(model_effi.to('cuda'), (3, 244, 244))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         ZeroPad2d-1          [-1, 3, 245, 245]               0
Conv2dStaticSamePadding-2         [-1, 32, 122, 122]             864
       BatchNorm2d-3         [-1, 32, 122, 122]              64
MemoryEfficientSwish-4         [-1, 32, 122, 122]               0
         ZeroPad2d-5         [-1, 32, 124, 124]               0
Conv2dStaticSamePadding-6         [-1, 32, 122, 122]             288
       BatchNorm2d-7         [-1, 32, 122, 122]              64
MemoryEfficientSwish-8         [-1, 32, 122, 122]               0
          Identity-9             [-1, 32, 1, 1]               0
Conv2dStaticSamePadding-10              [-1, 8, 1, 1]             264
MemoryEfficientSwish-11              [-1, 8, 1, 1]               0
         Identity-12              [-1, 8, 1, 1]               0
Conv2dStaticSamePadding-13             [-1, 32, 1, 1]             288
         I

In [26]:
keys = model_effi.state_dict().keys()
weights = model_effi.state_dict().values()
for (key, weight) in zip(keys, weights):
    print(key, weights.shape)

AttributeError: 'odict_values' object has no attribute 'shape'

In [ ]:
del_keys = ['_bn1.weight', '_bn1.bias', '_bn1.running_mean', '_bn1.running_var', '_bn1.num_batches_tracked', '_fc.weight', '_fc.bias']

In [29]:
for name, param in model_effi.named_parameters():
    print(name, param.shape)

_conv_stem.weight torch.Size([32, 3, 3, 3])
_bn0.weight torch.Size([32])
_bn0.bias torch.Size([32])
_blocks.0._depthwise_conv.weight torch.Size([32, 1, 3, 3])
_blocks.0._bn1.weight torch.Size([32])
_blocks.0._bn1.bias torch.Size([32])
_blocks.0._se_reduce.weight torch.Size([8, 32, 1, 1])
_blocks.0._se_reduce.bias torch.Size([8])
_blocks.0._se_expand.weight torch.Size([32, 8, 1, 1])
_blocks.0._se_expand.bias torch.Size([32])
_blocks.0._project_conv.weight torch.Size([16, 32, 1, 1])
_blocks.0._bn2.weight torch.Size([16])
_blocks.0._bn2.bias torch.Size([16])
_blocks.1._expand_conv.weight torch.Size([96, 16, 1, 1])
_blocks.1._bn0.weight torch.Size([96])
_blocks.1._bn0.bias torch.Size([96])
_blocks.1._depthwise_conv.weight torch.Size([96, 1, 3, 3])
_blocks.1._bn1.weight torch.Size([96])
_blocks.1._bn1.bias torch.Size([96])
_blocks.1._se_reduce.weight torch.Size([4, 96, 1, 1])
_blocks.1._se_reduce.bias torch.Size([4])
_blocks.1._se_expand.weight torch.Size([96, 4, 1, 1])
_blocks.1._se_expand

In [31]:
from torch import Tensor
class CustomEfficientNet(EfficientNet):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def _forward_impl(self, x: Tensor) -> Tensor:
        x = self.features(x)
        return x


In [32]:
model_effi = EfficientNet.from_name('efficientnet-b7')

TypeError: __init__() takes 1 positional argument but 3 were given